In [19]:
import os

os.environ["OMP_NUM_THREADS"] = "2"
os.environ["TF_NUM_INTRAOP_THREADS"] = "2"
os.environ["TF_NUM_INTEROP_THREADS"] = "2"

In [20]:
DATASET_PATH = "../../../sharedDir/dir_lcc_rn_01/Aves/Dataset_Aves"
TRAIN_DIR = f"{DATASET_PATH}/train"
VALID_DIR = f"{DATASET_PATH}/valid"
TEST_DIR  = f"{DATASET_PATH}/test"

IMG_SIZE = (160, 160)
BATCH_SIZE = 64

In [21]:
import numpy             as np
import matplotlib.pyplot as plt
import scipy

import pandas as pd
import tensorflow as tf
import tensorflowjs as tfjs
from matplotlib.pyplot import imshow

#from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [22]:
train_datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)

valid_datagen = ImageDataGenerator(rescale=1/255)
test_datagen  = ImageDataGenerator(rescale=1/255)

train_gen = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

valid_gen = valid_datagen.flow_from_directory(
    VALID_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

test_gen = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

NUM_CLASSES = train_gen.num_classes
print("Número de clases:", NUM_CLASSES)


Found 84635 images belonging to 525 classes.
Found 2625 images belonging to 525 classes.
Found 2625 images belonging to 525 classes.
Número de clases: 525


In [23]:
from keras.applications import MobileNetV2
from keras import layers, models, optimizers

base_model = MobileNetV2(
    include_top=False,
    weights="imagenet",
    input_shape=(160, 160, 3)
)

base_model.trainable = False

inputs = layers.Input(shape=(160, 160, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = models.Model(inputs, outputs)

model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 160, 160, 3)]     0         
                                                                 
 mobilenetv2_1.00_160 (Func  (None, 5, 5, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d_1  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dropout_1 (Dropout)         (None, 1280)              0         
                                                                 
 dense_1 (Dense)             (None, 525)               672525    
                                                                 
Total params: 2930509 (11.18 MB)
Trainable params: 672525 (

In [24]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [25]:
EPOCHS = 15

history = model.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=EPOCHS
)

Epoch 1/15
1323/1323 [==============================] - 307s 231ms/step - loss: 2.2954 - accuracy: 0.5053 - val_loss: 0.9367 - val_accuracy: 0.7764
Epoch 2/15
1323/1323 [==============================] - 307s 232ms/step - loss: 1.2808 - accuracy: 0.6846 - val_loss: 0.7483 - val_accuracy: 0.8088
Epoch 3/15
1323/1323 [==============================] - 298s 225ms/step - loss: 1.1188 - accuracy: 0.7187 - val_loss: 0.6823 - val_accuracy: 0.8202
Epoch 4/15
1323/1323 [==============================] - 300s 226ms/step - loss: 1.0425 - accuracy: 0.7349 - val_loss: 0.6369 - val_accuracy: 0.8320
Epoch 5/15
1323/1323 [==============================] - 292s 220ms/step - loss: 0.9961 - accuracy: 0.7438 - val_loss: 0.6286 - val_accuracy: 0.8350
Epoch 6/15
1323/1323 [==============================] - 298s 225ms/step - loss: 0.9497 - accuracy: 0.7570 - val_loss: 0.5960 - val_accuracy: 0.8484
Epoch 7/15
1323/1323 [==============================] - 298s 225ms/step - loss: 0.9237 - accuracy: 0.7610 - val_

In [26]:
base_model.trainable = True
fine_tune_at = 120 

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

In [27]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [28]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint("best_model.h5", save_best_only=True)
]

In [30]:
history = model.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=EPOCHS,
    callbacks=callbacks
)

Epoch 1/15
1323/1323 [==============================] - 391s 295ms/step - loss: 0.5373 - accuracy: 0.8499 - val_loss: 0.4575 - val_accuracy: 0.8865
Epoch 2/15


/home/lcc_rn_19/.conda/envs/aves/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1323/1323 [==============================] - 393s 297ms/step - loss: 0.4940 - accuracy: 0.8610 - val_loss: 0.4344 - val_accuracy: 0.8910
Epoch 3/15
1323/1323 [==============================] - 385s 291ms/step - loss: 0.4622 - accuracy: 0.8685 - val_loss: 0.4200 - val_accuracy: 0.8930
Epoch 4/15
1323/1323 [==============================] - 381s 288ms/step - loss: 0.4422 - accuracy: 0.8742 - val_loss: 0.3902 - val_accuracy: 0.8975
Epoch 5/15
1323/1323 [==============================] - 396s 300ms/step - loss: 0.4172 - accuracy: 0.8808 - val_loss: 0.4029 - val_accuracy: 0.8987
Epoch 6/15
1323/1323 [==============================] - 386s 292ms/step - loss: 0.4029 - accuracy: 0.8842 - val_loss: 0.3663 - val_accuracy: 0.9006
Epoch 7/15
1323/1323 [==============================] - 389s 294ms/step - loss: 0.3868 - accuracy: 0.8878 - val_loss: 0.3905 - val_accuracy: 0.8983
Epoch 8/15
1323/1323 [==============================] - 390s 294ms/step - loss: 0.3770 - accuracy: 0.8922 - val_loss: 0.371

In [31]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

val_loss, val_acc = model.evaluate(valid_gen)
test_loss, test_acc = model.evaluate(test_gen)

print("Validación → Loss:", val_loss, "Accuracy:", val_acc)
print("Test → Loss:", test_loss, "Accuracy:", test_acc)

y_pred = np.argmax(model.predict(test_gen), axis=1)
y_true = test_gen.classes
labels = list(test_gen.class_indices.keys())

print(classification_report(y_true, y_pred, target_names=labels))

42/42 [==============================] - 13s 312ms/step - loss: 0.2407 - accuracy: 0.9299
Validación → Loss: 0.33813509345054626 Accuracy: 0.9120000004768372
Test → Loss: 0.2406531274318695 Accuracy: 0.9299047589302063
42/42 [==============================] - 13s 308ms/step
                               precision    recall  f1-score   support

              ABBOTTS BABBLER       1.00      0.60      0.75         5
                ABBOTTS BOOBY       1.00      0.60      0.75         5
   ABYSSINIAN GROUND HORNBILL       1.00      1.00      1.00         5
        AFRICAN CROWNED CRANE       1.00      0.80      0.89         5
       AFRICAN EMERALD CUCKOO       1.00      1.00      1.00         5
            AFRICAN FIREFINCH       1.00      1.00      1.00         5
       AFRICAN OYSTER CATCHER       1.00      1.00      1.00         5
        AFRICAN PIED HORNBILL       1.00      0.80      0.89         5
          AFRICAN PYGMY GOOSE       1.00      1.00      1.00         5
              

/home/lcc_rn_19/.conda/envs/aves/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lcc_rn_19/.conda/envs/aves/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lcc_rn_19/.conda/envs/aves/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

In [32]:
model.save("aves_model.h5")
print("Modelo guardado como aves_model.h5")

Modelo guardado como aves_model.h5


/home/lcc_rn_19/.conda/envs/aves/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [33]:
tfjs_target_dir = "tfjs_model"
tfjs.converters.save_keras_model(model, tfjs_target_dir)
print("Exportado a TensorFlow.js en carpeta tfjs_model/")

Exportado a TensorFlow.js en carpeta tfjs_model/


In [34]:
model.save("saved_model_aves", save_format="tf")
print("Guardado en formato SavedModel.")

INFO:tensorflow:Assets written to: saved_model_aves/assets


INFO:tensorflow:Assets written to: saved_model_aves/assets


Guardado en formato SavedModel.


In [40]:
model.output_shape

(None, 525)

In [41]:
import os

clases = sorted(os.listdir(TRAIN_DIR))

print("Total de clases encontradas:", len(clases))
for c in clases:
    print(c)

with open("clases_aves.txt", "w") as f:
    for c in clases:
        f.write(c + "\n")

Total de clases encontradas: 525
ABBOTTS BABBLER
ABBOTTS BOOBY
ABYSSINIAN GROUND HORNBILL
AFRICAN CROWNED CRANE
AFRICAN EMERALD CUCKOO
AFRICAN FIREFINCH
AFRICAN OYSTER CATCHER
AFRICAN PIED HORNBILL
AFRICAN PYGMY GOOSE
ALBATROSS
ALBERTS TOWHEE
ALEXANDRINE PARAKEET
ALPINE CHOUGH
ALTAMIRA YELLOWTHROAT
AMERICAN AVOCET
AMERICAN BITTERN
AMERICAN COOT
AMERICAN DIPPER
AMERICAN FLAMINGO
AMERICAN GOLDFINCH
AMERICAN KESTREL
AMERICAN PIPIT
AMERICAN REDSTART
AMERICAN ROBIN
AMERICAN WIGEON
AMETHYST WOODSTAR
ANDEAN GOOSE
ANDEAN LAPWING
ANDEAN SISKIN
ANHINGA
ANIANIAU
ANNAS HUMMINGBIRD
ANTBIRD
ANTILLEAN EUPHONIA
APAPANE
APOSTLEBIRD
ARARIPE MANAKIN
ASHY STORM PETREL
ASHY THRUSHBIRD
ASIAN CRESTED IBIS
ASIAN DOLLARD BIRD
ASIAN GREEN BEE EATER
ASIAN OPENBILL STORK
AUCKLAND SHAQ
AUSTRAL CANASTERO
AUSTRALASIAN FIGBIRD
AVADAVAT
AZARAS SPINETAIL
AZURE BREASTED PITTA
AZURE JAY
AZURE TANAGER
AZURE TIT
BAIKAL TEAL
BALD EAGLE
BALD IBIS
BALI STARLING
BALTIMORE ORIOLE
BANANAQUIT
BAND TAILED GUAN
BANDED BROADBILL
BAN